In [1]:
import numpy as np
import pandas as pd


In [2]:
import os
import pandas as pd
from PIL import Image
import numpy as np

# Define the directory containing the images
image_dir = "/kaggle/input/utkface-new/UTKFace"

# Initialize lists to store data
data = {
    "img_name": [],
    "age": [],
    "gender": [],
    "ethnicity": [],
    "pixels": []
}

# Loop over all images in the directory
for img_name in os.listdir(image_dir):
    if img_name.endswith('.jpg.chip.jpg'):
        # Extract information from the filename
#         print(img_name.split('_'))
        age = img_name.split('_')[0]
        gender = img_name.split('_')[1]
        ethnicity = img_name.split('_')[2]
        # Open the image and convert it to grayscale
        img_path = os.path.join(image_dir, img_name)
        img = Image.open(img_path).convert('L')  # Convert to grayscale
        
        # Resize or reshape image if needed (optional)
        img = img.resize((48, 48))  # Example resize to 48x48
        pixels = np.array(img).flatten().tolist()  # Convert image to a flat list of pixel values
        
        # Append data to the lists
        data["img_name"].append(img_name)
        data["age"].append(int(age))
        data["gender"].append(int(gender))
        if len(ethnicity) == 1:
            data["ethnicity"].append(int(ethnicity))
        else:
            data["ethnicity"].append(-1)
        data["pixels"].append(' '.join(map(str, pixels)))

# Create a DataFrame from the data
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('age_gender.csv', index=False)


In [3]:
df = pd.read_csv('/kaggle/working/age_gender.csv')

In [4]:
data = df.drop(['img_name','age','ethnicity'],axis= 1)
data

,gender,pixels
0,0,7 4 6 5 14 75 146 172 183 185 189 193 196 200 ...
1,1,4 1 2 7 9 13 20 27 34 42 47 56 76 90 99 108 11...
2,1,37 35 32 28 24 20 23 32 38 45 57 72 86 99 112 ...
3,0,114 124 125 126 134 146 156 162 165 162 158 15...
4,1,115 129 143 160 172 178 184 188 191 194 195 19...
...,...,...
23702,0,46 53 67 90 101 103 103 107 114 117 120 123 12...
23703,0,213 198 177 135 120 92 75 72 68 65 64 64 53 54...
23704,0,190 190 191 193 197 200 201 202 204 204 206 20...
23705,1,122 139 116 141 163 161 158 142 128 126 152 17...


In [5]:
data['pixels']=data['pixels'].apply(lambda x:  np.array(x.split(), dtype="float32"))


data['pixels'] = data['pixels'].apply(lambda x: x/255)
data

,gender,pixels
0,0,"[0.02745098, 0.015686275, 0.023529412, 0.01960..."
1,1,"[0.015686275, 0.003921569, 0.007843138, 0.0274..."
2,1,"[0.14509805, 0.13725491, 0.1254902, 0.10980392..."
3,0,"[0.44705883, 0.4862745, 0.49019608, 0.49411765..."
4,1,"[0.4509804, 0.5058824, 0.56078434, 0.627451, 0..."
...,...,...
23702,0,"[0.18039216, 0.20784314, 0.2627451, 0.3529412,..."
23703,0,"[0.8352941, 0.7764706, 0.69411767, 0.5294118, ..."
23704,0,"[0.74509805, 0.74509805, 0.7490196, 0.75686276..."
23705,1,"[0.47843137, 0.54509807, 0.45490196, 0.5529412..."


In [6]:
data = data.to_numpy()



In [7]:

img = data[:,1:]

data_2d = np.vstack(img)

X = np.array([x[0] for x in img])
# Print the resulting 2D NumPy array
y = data[:, 0].astype(int)
# from sklearn.decomposition import PCA

# # Create a PCA instance with the desired number of components
# n_components = 500 # Adjust this based on the number of components you want to keep
# pca = PCA(n_components=n_components)
# # Fit the PCA model to your data and transform it
# X = pca.fit_transform(X)

X.shape

(23707, 2304)

In [8]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

k = 20 # Number of neighbors
knn_classifier = KNeighborsClassifier(n_neighbors=k)

knn_classifier.fit(X_train, y_train)

y_pred = knn_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)
print("Confusion Matrix:\n", conf_matrix)


Accuracy: 0.7372416701813581
Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.85      0.77      2490
           1       0.79      0.61      0.69      2252

    accuracy                           0.74      4742
   macro avg       0.75      0.73      0.73      4742
weighted avg       0.75      0.74      0.73      4742

Confusion Matrix:
 [[2128  362]
 [ 884 1368]]


In [9]:
y = data[0,:]
y

array([0,
       array([0.02745098, 0.01568628, 0.02352941, ..., 0.04313726, 0.01176471,
              0.01568628], dtype=float32)                                     ],
      dtype=object)